# ML testing: experiment #3-reqd harm (open nested combat)

This notebook involves testing for the clinical harmonization paper. This notebook shows mixed_dataset (TOP + StrokeMRI) based models with OPNested ComBat harmonization to Insight 46 and SABRE datasets (the StrokeMRI and TOP become one dataset)

Data: StrokeMRI, TOP, SABRE, Insight46, EDIS, HELIUS 

Harmonisation: Open nested combat

Training data: Norment whichh is StrokeMRI and TOP togehter

Testing data: test set from Norment

Futher data applied to: SABRE, Insight46, EDIS, (HELIUS pending)

Validation method: K-fold, double-stratified

Brain-age algorithms: LR, lasso, extra trees, elasticCV net fully tested (but not optimized parameters), additionals partly

Outputs: sent to output folder, predicted versus real dataframes and other artifacts

### import libraries

In [ ]:
import os       # using operating system dependent functionality (folders)
import sys

import glob
from functools import reduce

import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import matplotlib.pyplot as plt

# # demo stuff
# # demo stuff
import ipywidgets as widgets
from ipywidgets import interactive

# ml stuff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import tree
from sklearn import metrics
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score

import joblib

sys.path.insert(0, '../') # path to functions
import cvasl.seperated as sep

### import data

In [ ]:
filepath_mri_for_ids = 'our_datasets/StrokeMRI/'
filename_mri_for_ids = os.path.join(filepath_mri_for_ids,'TrainingDataComplete.csv') 

filepath_top_for_ids = 'our_datasets/TOP/' 
filename_top_for_ids = os.path.join(filepath_top_for_ids,'TrainingDataComplete.csv') 

filename_topmri = 'topmri_5_opn_harmonized_to_sab.csv'
filename_sabre ='sabre_5_opn_harmonized.csv'
filename_edis ='edis_5_opn_harmonized.csv'
filename_helius ='helius_5_opn_harmonized.csv'
filename_insight46 = 'insight_5_opn_harmonized.csv'
# read in data


filepath_ = 'harmonizations/harm_results/open_nested_combat'
#TOP_file = os.path.join(TOP_path, file_name)
#MRI_file = os.path.join(filepath_ , file_name_mri)
EDIS_file = os.path.join(filepath_ , filename_edis)
HELIUS_file = os.path.join(filepath_ , filename_helius)
Insight46_file = os.path.join(filepath_ , filename_insight46)
SABRE_file = os.path.join(filepath_ , filename_sabre)
TOPMRI_file = os.path.join(filepath_ , filename_topmri)

EDIS = pd.read_csv(EDIS_file, index_col=0)
HELIUS = pd.read_csv(HELIUS_file, index_col=0)
Insight46 = pd.read_csv(Insight46_file, index_col=0)
SABRE = pd.read_csv(SABRE_file, index_col=0)
#TOP_n = pd.read_csv(TOP_file, index_col=0)
#MRI_n = pd.read_csv(MRI_file, index_col=0)
TOPMRI = pd.read_csv(TOPMRI_file, index_col=0)

IDS_TOP =  pd.read_csv(filename_top_for_ids)
IDS_MRI =  pd.read_csv(filename_mri_for_ids)

In [ ]:
# Now we need to flip the sex back to numbers for a correlation
# sex_mapping = {'F':0,'M':1}
# TOPMRI = TOPMRI.assign(sex = TOPMRI.sex.map(sex_mapping))
TOPMRI.head(3)

In [ ]:
TOPMRI.columns

In [ ]:
#Insight46 = Insight46.assign(sex = Insight46.sex.map(sex_mapping))
Insight46.head(3)

## a few data quality checks

In [ ]:
datasets = [EDIS, SABRE, Insight46, TOPMRI, HELIUS]
for qset in datasets:
    print(qset.sex.unique())
    print(len(qset.columns))
    print(qset.isna().sum().sum())

In [ ]:
# # put columns to lower
# TOPMRI.columns = TOPMRI.columns.str.lower() 
# Insight46.columns = Insight46.columns.str.lower() 
# EDIS.columns = EDIS.columns.str.lower() 
# SABRE.columns= SABRE.columns.str.lower() 
# HELIUS.columns= HELIUS.columns.str.lower() 

In [ ]:
#give a checkbox for out put folder
loged_feat = widgets.ToggleButton(
    value=False,
    description='Click me if some features logged',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
loged_feat

In [ ]:
loged_feat.value

In [ ]:
if loged_feat.value == False:
    output_folder = '3c_no_log_outputs'
else:
    output_folder = '3c_loged_outputs'

os.makedirs(output_folder, exist_ok=True)

# Now we need to break up the top and MRI datasets as well, and format them like the others

In [ ]:
set_top_ids = set(IDS_TOP.participant_id)
set_mri_ids = set(IDS_MRI.participant_id)
StrokeMRI = TOPMRI[TOPMRI['participant_id'].isin(list(set_mri_ids))]
TOP = TOPMRI[TOPMRI['participant_id'].isin(list(set_top_ids))]
TOP.head(3)

In [ ]:
StrokeMRI.head(3)

In [ ]:
TOP.head(3)

In [ ]:
# check for any duplicated patients between stroke and mri
strokers = set(StrokeMRI.participant_id)
topers = set(TOP.participant_id)
z = strokers.intersection(topers)
print(z) 

In [ ]:
mixed_data = TOPMRI

In [ ]:

TOPMRI

## Build ML models

In [ ]:
mixed_data.columns

In [ ]:
#mixed_data = mixed_data.drop()

In [ ]:
ml_matrix = mixed_data #.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
y = ml_matrix['age'].values
y=y.astype('float')


In [ ]:
linr_k_frame, linr_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('linear regression', 'opnharm_mix_linr', LinearRegression(), ml_matrix, X[:,1:], y)

In [ ]:
linr_k_frame

In [ ]:
linr_k_frame.to_csv(output_folder +'linr_k_frame_opn_harm.csv')

In [ ]:
avg_linr = sep.avg_k_folds(linr_k_frame)
avg_linr

In [ ]:
linr_y_frame

In [ ]:
linr_y_frame.to_csv(output_folder + 'linr_y_frame_opn_harm.csv')

In [ ]:
linr = models[0]
linr[0]

In [ ]:
# check if model folder exists and if not , then create
model_folder = '../result_models/'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [ ]:
## optional save models
#joblib.dump(linr[0], ('../result_models/'+ 'opnharm_mix_linr0.sav'))
#joblib.dump(linr[1], ('../result_models/'+ 'opnharm_mix_linr1.sav'))
#joblib.dump(linr[2], ('../result_models/'+ 'opnharm_mix_linr2.sav'))
#joblib.dump(linr[3], ('../result_models/'+ 'opnharm_mix_linr3.sav'))
#joblib.dump(linr[4], ('../result_models/'+ 'opnharm_mix_linr4.sav'))

In [ ]:
llreg_k_frame, llreg_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('lasso regression', 'opnharm_mix_llreg',  linear_model.LassoLars(alpha=0.01), ml_matrix, X[:,1:], y)
llreg_k_frame

In [ ]:
llreg_k_frame.to_csv(output_folder + 'llreg_k_frame_opn_harm.csv')

In [ ]:
avg_llreg = sep.avg_k_folds(llreg_k_frame)
avg_llreg

In [ ]:
llreg_y_frame

In [ ]:
llreg_y_frame.to_csv(output_folder + 'llreg_y_frame_opn_harm.csv')

In [ ]:
llreg = models[0]
llreg[0]

In [ ]:
## optional save models
#joblib.dump(llreg[0], ('../result_models/'+ 'opnharm_mix_linr0.sav'))
#joblib.dump(llreg[1], ('../result_models/'+ 'opnharm_mix_linr1.sav'))
#joblib.dump(llreg[2], ('../result_models/'+ 'opnharm_mix_linr2.sav'))
#joblib.dump(llreg[3], ('../result_models/'+ 'opnharm_mix_linr3.sav'))
#joblib.dump(llreg[4], ('../result_models/'+ 'opnharm_mix_linr4.sav'))

In [ ]:
dtree_k_frame, dtree_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('decision tree', 'opnharm_mix_dtree',  tree.DecisionTreeRegressor(), ml_matrix, X[:,1:], y)
dtree_k_frame

In [ ]:
dtree_k_frame.to_csv(output_folder + 'dtree_k_frame_opn_harm.csv')

In [ ]:
avg_dtree = sep.avg_k_folds(dtree_k_frame)
avg_dtree

In [ ]:
dtree_y_frame

In [ ]:
dtree_y_frame.to_csv(output_folder + 'dtree_y_frame_opn_harm.csv')

In [ ]:
dtree = models[0]
dtree[0]

In [ ]:
regr_k_frame, regr_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('MLP regression', 'opnharm_mix_regr',   MLPRegressor(random_state=1, max_iter=700), ml_matrix, X[:,1:], y)
regr_k_frame

In [ ]:
regr_k_frame.to_csv( output_folder +'regr_k_frame_opn_harm.csv')

In [ ]:
avg_regr = sep.avg_k_folds(regr_k_frame)
avg_regr

In [ ]:
regr_y_frame

In [ ]:
regr_y_frame.to_csv(output_folder + 'regr_y_frame_opn_harm.csv')

In [ ]:
regr = models[0]
regr[0]

In [ ]:
svrp2_k_frame, svrp2_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('support vector reg poly2', 'opnharm_mix_svrp2',   SVR(C=1.0, kernel='poly', degree =2, epsilon=0.2), ml_matrix, X[:,1:], y)
svrp2_k_frame

In [ ]:
svrp2_k_frame.to_csv(output_folder + 'svrp2_k_frame_opn_harm.csv')

In [ ]:
avg_svrp2 = sep.avg_k_folds(svrp2_k_frame)
avg_svrp2

In [ ]:
svrp2_y_frame

In [ ]:
svrp2_y_frame.to_csv(output_folder + 'svrp2_y_frame_opn_harm.csv')

In [ ]:
svrp2 = models[0]
svrp2[0]

In [ ]:
eregr_k_frame, eregr_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('elasticnetCV', 'opnharm_mix_eregr',  ElasticNetCV(cv=5, random_state=12), ml_matrix, X[:,1:], y)
eregr_k_frame

In [ ]:
eregr_k_frame.to_csv(output_folder + 'eregr_k_frame_opn_harm.csv')

In [ ]:
avg_eregr = sep.avg_k_folds(eregr_k_frame)
avg_eregr

In [ ]:
eregr_y_frame

In [ ]:
eregr_y_frame.to_csv(output_folder + 'eregr_y_frame_opn_harm.csv')

In [ ]:
eregr = models[0]
eregr[0]

In [ ]:
etreg_k_frame, etreg_y_frame, models = sep.stratified_cat_and_cont_categories_shuffle_split('extra trees', 'opnharm_mix_etreg',  ExtraTreesRegressor(n_estimators=100, random_state=0), ml_matrix, X[:,1:], y)
etreg_k_frame

In [ ]:
etreg_k_frame.to_csv(output_folder + 'etreg_k_frame_opn_harm.csv')

In [ ]:
avg_etreg = sep.avg_k_folds(etreg_k_frame)
avg_etreg

In [ ]:
etreg_y_frame

In [ ]:
etreg_y_frame.to_csv(output_folder + 'etreg_y_frame_opn_harm.csv')

In [ ]:
etreg = models[0]
etreg[0]

In [ ]:
## optional save models
#joblib.dump(etreg[0], ('../result_models/'+ 'opnharm_mix_etreg0.sav'))
#joblib.dump(etreg[1], ('../result_models/'+ 'opnharm_mix_etreg1.sav'))
#joblib.dump(etreg[2], ('../result_models/'+ 'opnharm_mix_etreg2.sav'))
#joblib.dump(etreg[3], ('../result_models/'+ 'opnharm_mix_etreg3.sav'))
#joblib.dump(etreg[4], ('../result_models/'+ 'opnharm_mix_etreg4.sav'))

Show results ON AVERAGE for each model

In [ ]:
mixed_based_opnharmonized_on_testmix =pd.concat([avg_linr,
                   avg_llreg,
                   avg_dtree,
                   avg_regr,
                   avg_svrp2,
                   avg_eregr,
                   avg_etreg],
                  axis=0)
mixed_based_opnharmonized_on_testmix

In [ ]:
mixed_based_opnharmonized_on_testmix.to_csv(output_folder + 'mixed_based_opnharmonized_on_testmix.csv')

# Running mixed model over SABRE dataset

## Here we will do an example of running  models made of the entire StrokeMRI and TOP dataset mixed as one

#### Build new models

In [ ]:
TOPMRI.head(3)

In [ ]:
TOPMRI = TOPMRI.drop(['binned', 'fuse_bin'], axis=1)
TOPMRI

In [ ]:
ml_matrix = TOPMRI.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
X = X.astype('float')
y = ml_matrix['age'].values
y=y.astype('float')


In [ ]:
# now we decide to do no test train split, rather take all
X_train = X
y_train = y

In [ ]:
MIXlinr = LinearRegression()
MIXlinr.fit(X_train, y_train)

In [ ]:
MIXllreg = linear_model.LassoLars(alpha=0.01)
MIXllreg.fit(X_train, y_train)

In [ ]:
MIXeregr = ElasticNetCV(cv=5, random_state=17)
MIXeregr.fit(X_train, y_train)


In [ ]:
MIXetreg = ExtraTreesRegressor(n_estimators=100, random_state=0)
MIXetreg.fit(X_train, y_train)

##  Save these four best models

In [ ]:

## optional save models
#joblib.dump(MIXlinr, ('../result_models/'+  'opn_harm_mix_MIXlinr.sav'))
#joblib.dump(MIXllreg, ('../result_models/'+ 'opn_harmm_mix_MIXllreg.sav'))

#joblib.dump(MIXeregr, ('../result_models/'+ 'opn_harm_mix_MIXeregr.sav'))
#joblib.dump(MIXetreg, ('../result_models/'+ 'opn_harm_mix_MIXetreg.sav'))

# Run models on SABRE

In [ ]:
sabre_ml_matrix = SABRE.drop('participant_id', axis=1)
X_sabre = sabre_ml_matrix.drop('age', axis =1)
X_sabre = X_sabre.values
X_sabre = X_sabre.astype('float')
y_sabre = sabre_ml_matrix['age'].values
y_sabre=y_sabre.astype('float')

In [ ]:
X_sabre_test = X_sabre
y_sabre_test = y_sabre

In [ ]:
y_sabre_pred = MIXlinr.predict(X_sabre_test)

In [ ]:
data= [[
    'Linear Reg',
    'opn_harm_mix_MIXlinr.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXlinr.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
linr_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_sabre

In [ ]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_linr_sabre = pd.DataFrame(data)
y_frame_linr_sabre

In [ ]:
y_frame_linr_sabre.to_csv(output_folder + 'y_frame_linr_sabre_opn_harm.csv')

In [ ]:
y_sabre_pred = MIXllreg.predict(X_sabre_test)

In [ ]:
data= [[
    'Lasso',
    'opn_harmm_mix_MIXllreg.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXllreg.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
llreg_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_sabre

In [ ]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_llreg_sabre = pd.DataFrame(data)
y_frame_llreg_sabre

In [ ]:
y_frame_llreg_sabre.to_csv(output_folder + 'y_frame_llreg_sabre_opn_harm.csv')

In [ ]:
y_sabre_pred = MIXeregr.predict(X_sabre_test)

In [ ]:
data= [[
    'ElasticnetCV',
    'opn_harmm_mix_MIXeregr.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXeregr.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
eregr_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_sabre

In [ ]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_eregr_sabre = pd.DataFrame(data)
y_frame_eregr_sabre

In [ ]:
y_frame_eregr_sabre.to_csv(output_folder + 'y_frame_eregr_sabre_opn_harm.csv')

In [ ]:
y_sabre_pred = MIXetreg.predict(X_sabre_test)

In [ ]:
data= [[
    'Extra trees',
    'opn_harmm_mix_MIXetreg.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXetreg.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
etreg_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_sabre

In [ ]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_etregr_sabre = pd.DataFrame(data)
y_frame_etregr_sabre

In [ ]:
y_frame_etregr_sabre.to_csv(output_folder + 'y_frame_etregr_sabre_opn_harm.csv')

In [ ]:
mix_based_opnharmonized_on_sabre =pd.concat([linr_results_sabre,
                   llreg_results_sabre,
                   eregr_results_sabre,
                  etreg_results_sabre],
                  axis=0)
mix_based_opnharmonized_on_sabre

In [ ]:
mix_based_opnharmonized_on_sabre.to_csv(output_folder + 'mix_based_opnharmonized_on_sabre_opn_harm.csv')

# Running mixed model over Insight46 dataset

In [ ]:
insight_ml_matrix = Insight46.drop('participant_id', axis=1)
X_insight = insight_ml_matrix.drop('age', axis =1)
X_insight = X_insight.values
X_insight = X_insight.astype('float')
y_insight = insight_ml_matrix['age'].values
y_insight= y_insight.astype('float')

In [ ]:
X_insight_test = X_insight
y_insight_test = y_insight

In [ ]:
y_insight_pred = MIXlinr.predict(X_insight_test)

In [ ]:
data= [[
    'Linear Reg',
    'opn_harmm_mix_MIXllinr.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXlinr.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
linr_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_insight

In [ ]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_linr_insight = pd.DataFrame(data)
y_frame_linr_insight

In [ ]:
y_frame_linr_insight.to_csv(output_folder + 'y_frame_linr_insight_opn_harm.csv')

In [ ]:
y_insight_pred = MIXllreg.predict(X_insight_test)

In [ ]:
data= [[
    'Lasso',
    'opn_harmm_mix_MIXllreg.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXllreg.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
llreg_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_insight

In [ ]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_llreg_insight = pd.DataFrame(data)
y_frame_llreg_insight

In [ ]:
y_frame_llreg_insight.to_csv(output_folder + 'y_frame_llreg_insight_opn_harm.csv')

In [ ]:
y_insight_pred = MIXeregr.predict(X_insight_test)

In [ ]:
data= [[
    'ElasticnetCV',
    'opn_harmm_mix_MIXeregr.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXeregr.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
eregr_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_insight

In [ ]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_eregr_insight = pd.DataFrame(data)
y_frame_eregr_insight

In [ ]:
y_frame_eregr_insight.to_csv(output_folder + 'y_frame_eregr_insight_opn_harm.csv')

In [ ]:
y_insight_pred = MIXetreg.predict(X_insight_test)

In [ ]:
data= [[
    'Extra trees',
    'opnharm_mix_etreg.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXetreg.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
etreg_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_insight

In [ ]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_etreg_insight = pd.DataFrame(data)
y_frame_etreg_insight

In [ ]:
y_frame_etreg_insight.to_csv(output_folder + 'y_frame_etreg_insight_opn_harm.csv')

In [ ]:
mix_based_opnharmonized_on_insight =pd.concat([linr_results_insight,
                   llreg_results_insight,
                   eregr_results_insight,
                   etreg_results_insight],
                  axis=0)
mix_based_opnharmonized_on_insight

In [ ]:
mix_based_opnharmonized_on_insight.to_csv(output_folder + 'mix_based_opnharmonized_on_insight.csv')

# Run models on EDIS

In [ ]:
edis_ml_matrix = EDIS.drop('participant_id', axis=1)
X_edis = edis_ml_matrix.drop('age', axis =1)
X_edis = X_edis.values
X_edis = X_edis.astype('float')
y_edis = edis_ml_matrix['age'].values
y_edis=y_edis.astype('float')

In [ ]:
X_edis_test = X_edis
y_edis_test = y_edis

In [ ]:
y_edis_pred = MIXlinr.predict(X_edis_test)

In [ ]:
data= [[
    'Linear Reg',
    'opn_harm_mix_MIXlinr.sav',
    mean_absolute_error(y_edis_test, y_edis_pred),
    MIXlinr.score(X_edis_test,y_edis_test),
    metrics.explained_variance_score(y_edis_test, y_edis_pred)]]
linr_results_edis = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_edis

In [ ]:
data = {'real': y_edis_test, 'predicted': y_edis_pred}
y_frame_linr_edis = pd.DataFrame(data)
y_frame_linr_edis

In [ ]:
y_frame_linr_edis.to_csv(output_folder + 'y_frame_linr_edis_opn_harm.csv')

In [ ]:
y_edis_pred = MIXllreg.predict(X_edis_test)

In [ ]:
data= [[
    'Lasso',
    'opn_harmm_mix_MIXllreg.sav',
    mean_absolute_error(y_edis_test, y_edis_pred),
    MIXllreg.score(X_edis_test,y_edis_test),
    metrics.explained_variance_score(y_edis_test, y_edis_pred)]]
llreg_results_edis = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_edis

In [ ]:
data = {'real': y_edis_test, 'predicted': y_edis_pred}
y_frame_llreg_edis = pd.DataFrame(data)
y_frame_llreg_edis

In [ ]:
y_frame_llreg_edis.to_csv(output_folder + 'y_frame_llreg_edis_opn_harm.csv')

In [ ]:
y_edis_pred = MIXeregr.predict(X_edis_test)

In [ ]:
data= [[
    'ElasticnetCV',
    'opn_harmm_mix_MIXeregr_edis.sav',
    mean_absolute_error(y_edis_test, y_edis_pred),
    MIXeregr.score(X_edis_test,y_edis_test),
    metrics.explained_variance_score(y_edis_test, y_edis_pred)]]
eregr_results_edis = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_edis

In [ ]:
data = {'real': y_edis_test, 'predicted': y_edis_pred}
y_frame_eregr_edis = pd.DataFrame(data)
y_frame_eregr_edis

In [ ]:
y_frame_eregr_edis.to_csv(output_folder + 'y_frame_eregr_edis_opn_harm.csv')

In [ ]:
y_edis_pred = MIXetreg.predict(X_edis_test)

In [ ]:
data= [[
    'Extra trees',
    'opn_harmm_mix_MIXetreg_edis.sav',
    mean_absolute_error(y_edis_test, y_edis_pred),
    MIXetreg.score(X_edis_test,y_edis_test),
    metrics.explained_variance_score(y_edis_test, y_edis_pred)]]
etreg_results_edis = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_edis

In [ ]:
data = {'real': y_edis_test, 'predicted': y_edis_pred}
y_frame_etregr_edis = pd.DataFrame(data)
y_frame_etregr_edis

In [ ]:
y_frame_etregr_edis.to_csv(output_folder + 'y_frame_etregr_edis_opn_harm.csv')

In [ ]:
mix_based_opnharmonized_on_edis =pd.concat([linr_results_edis,
                   llreg_results_edis,
                   eregr_results_edis,
                  etreg_results_edis],
                  axis=0)
mix_based_opnharmonized_on_edis

In [ ]:
mix_based_opnharmonized_on_edis.to_csv(output_folder + 'mix_based_opnharmonized_on_edis_opn_harm.csv')

# Run models on HELIUS

In [ ]:
helius_ml_matrix = HELIUS.drop('participant_id', axis=1)
X_helius = helius_ml_matrix.drop('age', axis =1)
X_helius = X_helius.values
X_helius = X_helius.astype('float')
y_helius = helius_ml_matrix['age'].values
y_helius=y_helius.astype('float')

In [ ]:
X_helius_test = X_helius
y_helius_test = y_helius

In [ ]:
y_helius_pred = MIXlinr.predict(X_helius_test)

In [ ]:
data= [[
    'Linear Reg',
    'opn_harm_mix_MIXlinr.sav',
    mean_absolute_error(y_helius_test, y_helius_pred),
    MIXlinr.score(X_helius_test,y_helius_test),
    metrics.explained_variance_score(y_helius_test, y_helius_pred)]]
linr_results_helius = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_helius

In [ ]:
data = {'real': y_helius_test, 'predicted': y_helius_pred}
y_frame_linr_helius = pd.DataFrame(data)
y_frame_linr_helius

In [ ]:
y_frame_linr_helius.to_csv(output_folder + 'y_frame_linr_helius_opn_harm.csv')

In [ ]:
y_helius_pred = MIXllreg.predict(X_helius_test)

In [ ]:
data= [[
    'Lasso',
    'opn_harmm_mix_MIXllreg.sav',
    mean_absolute_error(y_helius_test, y_helius_pred),
    MIXllreg.score(X_helius_test,y_helius_test),
    metrics.explained_variance_score(y_helius_test, y_helius_pred)]]
llreg_results_helius = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_helius

In [ ]:
data = {'real': y_helius_test, 'predicted': y_helius_pred}
y_frame_llreg_helius = pd.DataFrame(data)
y_frame_llreg_helius

In [ ]:
y_frame_llreg_helius.to_csv(output_folder + 'y_frame_llreg_helius_opn_harm.csv')

In [ ]:
y_helius_pred = MIXeregr.predict(X_helius_test)

In [ ]:
data= [[
    'ElasticnetCV',
    'opn_harmm_mix_MIXeregr.sav',
    mean_absolute_error(y_helius_test, y_helius_pred),
    MIXeregr.score(X_helius_test,y_helius_test),
    metrics.explained_variance_score(y_helius_test, y_helius_pred)]]
eregr_results_helius = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_helius

In [ ]:
data = {'real': y_helius_test, 'predicted': y_helius_pred}
y_frame_eregr_helius = pd.DataFrame(data)
y_frame_eregr_helius

In [ ]:
y_frame_eregr_helius.to_csv(output_folder + 'y_frame_eregr_helius_opn_harm.csv')

In [ ]:
y_helius_pred = MIXetreg.predict(X_helius_test)

In [ ]:
data= [[
    'Extra trees',
    'opn_harmm_mix_MIXetreg.sav',
    mean_absolute_error(y_helius_test, y_helius_pred),
    MIXetreg.score(X_helius_test,y_helius_test),
    metrics.explained_variance_score(y_helius_test, y_helius_pred)]]
etreg_results_helius = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_helius

In [ ]:
data = {'real': y_helius_test, 'predicted': y_helius_pred}
y_frame_etregr_helius = pd.DataFrame(data)
y_frame_etregr_helius

In [ ]:
y_frame_etregr_helius.to_csv(output_folder + 'y_frame_etregr_helius_opn_harm.csv')

In [ ]:
mix_based_opnharmonized_on_helius =pd.concat([linr_results_helius,
                   llreg_results_helius,
                   eregr_results_helius,
                  etreg_results_helius],
                  axis=0)
mix_based_opnharmonized_on_helius

In [ ]:
mix_based_opnharmonized_on_helius.to_csv(output_folder + 'mix_based_opnharmonized_on_helius_opn_harm.csv')